题目$\frac{1}{x}+\frac{1}{y}<\frac{1}{n}$ ,根据题意可知n<x<=y

参考地址：https://blog.csdn.net/piaocoder/article/details/47954385

# 1. 用因子对表示解

先回忆已知等价式：

$$
\frac{1}{x} + \frac{1}{y} = \frac{1}{n}
\quad\Longleftrightarrow\quad
(x-n)(y-n)=n^2.
$$

令：

$$
a = x-n,\qquad b = y-n.
$$

注意 $x,y,n$ 都取正整数并且我们要求 $x>n,\; y>n$，所以 $a,b$ 都是正整数。  
于是每一对正整数因子 $(a,b)$ 满足 $ab=n^2$ 就对应一组解：

$$
(x,y)=(n+a,n+b).
$$

---

# 2. 有序对与无序对的关系

- $n^2$ 的正因子总数记作 $\tau(n^2)$.  
- $\tau(n^2)$ 精确等于满足 $ab=n^2$ 的 **有序** 正因子对 $(a,b)$ 的数量（因为每个因子 $d$ 对应 $(d, n^2/d)$）。  
- $(x,y)$ 视为 **不计顺序** 或者要求 $x \le y$. 把有序的因子对变为不计顺序的对，需要把互换顺序视为同一解。

---

# 3. 平方数的特殊性（中轴对）

一般地，把有序对数量转为不计顺序的数量，若没有自反对会是：

$$
\frac{\tau(n^2)}{2}.
$$

但当 $n^2$ 是完全平方时，存在自反对 $(\sqrt{n^2},\sqrt{n^2})=(n,n)$（即 $a=b$），它在有序对里只出现一次，在无序对里也只算一次。

因此总的不计顺序的因子对数（即满足 $a \le b$ 的对数）为：

$$
\#\{(a,b):\ ab=n^2,\; a \le b\}=\frac{\tau(n^2)+1}{2}.
$$

（直观：把所有 $\tau(n^2)$ 个有序对配成 $\tau(n^2)/2$ 对互换对，再加上中轴自反的那一个。）

---

# 4. 把“解的数量 = 4,000,000”转换成 $\tau(n^2)$ 的等式

题意给出：满足 $x \le y$ 的正整数解个数为 $4{,}000{,}000$. 根据上面等式：

$$
\frac{\tau(n^2)+1}{2}=4{,}000{,}000.
$$

解这个等式得：

$$
\tau(n^2)+1 = 8{,}000{,}000 \quad\Longrightarrow\quad \tau(n^2)=7{,}999{,}999.
$$

分解：  

$$
7,999,999 = 7 \times 199 \times 5743,
$$  

且这三个因子都是素数。 

---

## 5) 使 $n$ 最小的构造与排列原则  

给定 $(2e_i+1)$ 的乘积固定，要使  

$$
n=\prod_{i=1}^k p_i^{e_i}
$$  

最小，应将 **最大的指数配给最小的质数**。  
（重排不等式/贪心原则：大指数配小质数能最小化积。）  

把三个因子直接作为三个 $(2e_i+1)$：  

$$
(2e_1+1,\;2e_2+1,\;2e_3+1)=(5743,\;199,\;7),
$$  

对应：  

$$
(e_1,e_2,e_3)=\left(\frac{5743-1}{2},\;\frac{199-1}{2},\;\frac{7-1}{2}\right)=(2871,\;99,\;3).
$$  

将它们依大到小配给最小的质数 $2,3,5$：  

$$
n = 2^{2871} \cdot 3^{99} \cdot 5^3.
$$  

---

## 6) 校验  

$$
\tau(n^2) = (2\cdot 2871+1)(2\cdot 99+1)(2\cdot 3+1)
= 5743 \cdot 199 \cdot 7 = 7,999,999,
$$  

$$
\frac{\tau(n^2)+1}{2}=4,000,000.
$$  

---

# 7. （补充）为什么 $\tau(n^2)$ 总是奇数？

若

$$
n=\prod_i p_i^{e_i},
$$

则

$$
n^2=\prod_i p_i^{2e_i},\qquad \tau(n^2)=\prod_i (2e_i+1).
$$

每个因子 $2e_i+1$ 都是奇数，奇数的乘积仍为奇数。  
所以 $\tau(n^2)$ 必是奇数；因此 $\tau(n^2)=7{,}999{,}999$ 是合理的奇数目标值。




In [ ]:
import math
import time
from functools import lru_cache

# --- 简单试除法分解（对 ~1e7 量级非常快） ---
#从偶数2开始，3,5,7，不断拆分
def factorize_small(n):
    fac = {}
    d = 2
    while d * d <= n:
        while n % d == 0:
            fac[d] = fac.get(d, 0) + 1
            n //= d
        d += 1 if d == 2 else 2  # 2后只试奇数
    if n > 1:
        fac[n] = fac.get(n, 0) + 1
    return fac

def divisors_from_factor(factors):
    # factors: dict p->exp
    items = list(factors.items())
    divs = [1]
    for p, e in items:
        cur = []
        for a in divs:
            mul = 1
            for _ in range(e+1):
                cur.append(a * mul)
                mul *= p
        divs = cur
    return sorted(divs)

# --- 质数表（够用的前若干质数） ---
def first_primes(n):
    # 简单生成前 n 素数（n 不会太大）
    primes = []
    cand = 2
    while len(primes) < n:
        is_p = True
        r = int(math.isqrt(cand))
        for p in range(2, r+1):
            if cand % p == 0:
                is_p = False
                break
        if is_p:
            primes.append(cand)
        cand += 1
    return primes

# --- 主搜索 ---
def min_n_for_k(k, time_limit_sec=60):
    start_time = time.time()

    T = 2 * k - 1  # 必须为奇数
    # factorize T
    fac = factorize_small(T)
    # quick sanity
    if T == 1:
        return 1

    divs = divisors_from_factor(fac)
    # we only need odd divisors >= 3 (2e+1 >= 3)
    odd_divs = [d for d in divs if (d % 2 == 1 and d >= 3)]
    odd_divs.sort()

    # create set for fast membership
    div_set = set(odd_divs)
    #
    # primes: we don't know how many factors we end up with, but safe to take first 30 primes
    primes = first_primes(60)
    #
    best_n = None

    # If full generation feasible, do that but avoid extremely large explosion by early exit if time exceeded
    # We'll generate on the fly, converting each partition into exponents and compute n, keep minimal.
    # Instead of building all partitions in memory, iterate recursively:
    def dfs_build(remaining, start_index, current_factors):
        nonlocal best_n
        # time check
        if time.time() - start_time > time_limit_sec:
            raise TimeoutError("time limit exceeded")

        if remaining == 1:
            # evaluate this factor list
            # convert to exponents e_i = (d-1)//2, sort desc, assign to primes
            exps = sorted(((d - 1) // 2 for d in current_factors), reverse=True)
            # compute n
            n_val = 1
            for i, e in enumerate(exps):
                n_val *= pow(primes[i], e)
                # pruning: if best_n known and current partial product > best_n, can stop
                if best_n is not None and n_val >= best_n:
                    return
            if best_n is None or n_val < best_n:
                best_n = n_val
            return

        # iterate allowed factors starting from start_index in odd_divs
        for idx in range(start_index, len(odd_divs)):
            d = odd_divs[idx]
            if d > remaining:
                break
            if remaining % d != 0:
                continue

            # quick optimistic lower-bound pruning:
            # If we take d now, remaining' = remaining//d, we can estimate a lower bound for n by
            # converting current_factors + [d] to exps and assigning largest exps to smallest primes,
            # but because we don't know future splits, build a conservative LB by assuming the rest stays as 1 factor
            # (coarse). We'll instead do a looser check: compute partial product for current exponents and
            # if exceeds best_n, prune.
            new_factors = current_factors + (d,)

            # partial exps:
            partial_exps = sorted(((x - 1) // 2 for x in new_factors), reverse=True)
            partial_n = 1
            for i, e in enumerate(partial_exps):
                partial_n *= pow(primes[i], e)
                if best_n is not None and partial_n >= best_n:
                    break
            if best_n is not None and partial_n >= best_n:
                continue  # prune

            dfs_build(remaining // d, idx, new_factors)
            # time check in loop
            if time.time() - start_time > time_limit_sec:
                raise TimeoutError("time limit exceeded")

    # Kick off search
    try:
        dfs_build(T, 0, ())
    except TimeoutError as te:
        print("搜索超时，当前找到的最优解（如果有）将返回:", te)
    return best_n

In [ ]:
k =3
t0 = time.time()
result = min_n_for_k(k, time_limit_sec=120)  # 120 秒上限，可按需调
t1 = time.time()
print("k =", k)
print("最小 n =", result)
print("计算耗时: {:.3f} 秒".format(t1 - t0))

In [ ]:
k =1000
t0 = time.time()
result = min_n_for_k(k, time_limit_sec=120)  # 120 秒上限，可按需调
t1 = time.time()
print("k =", k)
print("最小 n =", result)
print("计算耗时: {:.3f} 秒".format(t1 - t0))

In [ ]:
k =4000000
t0 = time.time()
result = min_n_for_k(k, time_limit_sec=120)  # 120 秒上限，可按需调
t1 = time.time()
print("k =", k)
print("最小 n =", result)
print("计算耗时: {:.3f} 秒".format(t1 - t0))

In [2]:
import math
from numba import njit

@njit
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

@njit
def isqrt(n):
    """整数平方根，兼容 numba"""
    x = int(math.sqrt(n))
    while (x+1)*(x+1) <= n:
        x += 1
    while x*x > n:
        x -= 1
    return x

@njit
def generate_triples_count(L):
    total = 0
    j_max = (isqrt(1 + 4 * L) - 1) // 2

    for j in range(2, j_max + 1):
        i_upper_by_L = L // j - j
        if i_upper_by_L < 1:
            continue
        i_max = min(j - 1, i_upper_by_L)

        for i in range(1, i_max + 1):
            if gcd(i, j) != 1:
                continue
            denom = (i + j) * j
            k_max = L // denom
            if k_max >= 1:
                total += k_max

    return total

In [3]:
L = 10**9
print("L =", L, "→ count =", generate_triples_count(L))



L = 1000000000 → count = 3979600400


In [ ]:
# ⚠️ 第一次调用会有编译开销
# 第二次调用就会很快
L = 10**12
print("L =", L, "→ count =", generate_triples_count(L))